#  Introducción al IECore e IENetwork

Para poder interactuar con el Motor de Inferencia de Intel(Inference Engine) es necesario utilizar dos clases de Python, que son las siguientes:
- `IECore`
- `IENetwork`
Brevemente, se puede visualizar la forma de integración de OpenVINO en cualquier aplicación, básicamente comprende en los siguientes pasos, iniciando con la inicialización del IECore[1].
![IE CORE](https://docs.openvinotoolkit.org/latest/integration_process.png)
<div align="center"><b>Fig 1.</b> Pasos de la integración de OpenVINO en una aplicación[1]</div>
Ahora, se verá brevemente los por menores de ambas clases.

## IECore
Este elemento del `IECore`[2] es fundamental para la ejecución del Toolkit OpenVINO, es el puente con el Motor de Inferencia(IE) y permite interactuar con el Hardware, conocer la lista de dispositivos disponibles, activar Plugins de Harware, consultar la compatibilidad de las capas, etc. Para que poder realizar inferencia, necesita cargar un objeto de tipo `IE Network` que es el siguiente a tratar.


## IENetwork
Este elemento `IENetwork`[3] es el elemento que inicialmente inicializa nuestro modelo de Deep Learning y lo pasa al elemento del `IECore`, además este elemento permite conocer la configuración de entradas y salidas del modelo, nombre del modelo, entradas, etc. Se verá de mejor forma en el siguiente ejemplo.

## Ejemplo

### Descargar un modelo Pre-Entrenado con el Model Downloader
Para este ejemplo es necesario primero descargar el modelo `person-detection-retail-0013`[4] para detección de peatones. Por favor, **ejecutar** esta celda para descargar el archivo



In [1]:
import os, sys
nombre_modelo = "person-detection-retail-0013"
ruta_archivo = os.getcwd()
if sys.platform == "win32":
    ruta_model_downloader="C:\Program Files (x86)\Intel\openvino_2021\deployment_tools\tools\model_downloader\downloader.py"
    !python {ruta_model_downloader} --name {nombre_modelo}
    print("Ruta donde se guardó el archivo: {}".format(ruta_archivo))
elif sys.platform == "linux":
    ruta_model_downloader="/opt/intel/openvino_2021/deployment_tools/tools/model_downloader/downloader.py"
    !python {ruta_model_downloader} --name {nombre_modelo}
    print("Ruta donde se guardó el archivo: {}".format(ruta_archivo))

################|| Downloading person-detection-retail-0013 ||################

========== Downloading /home/josejacomeb/Documents/Workshop-ESPEL-MCT-2021/Sección 4/intel/person-detection-retail-0013/FP32/person-detection-retail-0013.xml
... 100%, 374 KB, 439 KB/s, 0 seconds passed

========== Downloading /home/josejacomeb/Documents/Workshop-ESPEL-MCT-2021/Sección 4/intel/person-detection-retail-0013/FP32/person-detection-retail-0013.bin
... 100%, 2823 KB, 485 KB/s, 5 seconds passed

========== Downloading /home/josejacomeb/Documents/Workshop-ESPEL-MCT-2021/Sección 4/intel/person-detection-retail-0013/FP16/person-detection-retail-0013.xml
... 100%, 373 KB, 429 KB/s, 0 seconds passed

========== Downloading /home/josejacomeb/Documents/Workshop-ESPEL-MCT-2021/Sección 4/intel/person-detection-retail-0013/FP16/person-detection-retail-0013.bin
... 100%, 1411 KB, 431 KB/s, 3 seconds passed

========== Downloading /home/josejacomeb/Documents/Workshop-ESPEL-MCT-2021/Sección 4/intel/person-dete

Dentro de la carpeta `intel/person-detection-retail-0013/` se han descargado tres modelos, con varias precisiones(`FP32, FP16, FP16-INT6`), los archivos que nos interesan son los siguientes:
- Archivo que contiene los pesos: `intel/person-detection-retail-0013/{Precision}/person-detection-retail-0013.bin`
- Archivo que contiene la arquitectura: `intel/person-detection-retail-0013/{Precision}/person-detection-retail-0013.xml`

### Inicializar las variables de entorno de OpenVINO
Como siempre, hay que inicializar las variables de entorno de OpenVINO, por favor **ejecutar** la celda correspondiente: 
- *Windows*

In [ ]:
%%cmd
"C:\Program Files (x86)\Intel\openvino_2021\bin\setupvars.bat"

- *Linux*

In [3]:
%%bash
source /opt/intel/openvino_2021/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


### Ejecutar el código de Python
En el presente código, vamos a inicializar los objetos de tipo `IECore`[2] y `IENetwork`[3], cargaremos el modelo y obtendremos información básica sobre el mismo

In [12]:
#Cargar las clases de IECore, IENetwork
from openvino.inference_engine import IECore

ruta_modelo_xml = "intel/person-detection-retail-0013/FP32/person-detection-retail-0013.xml"
ruta_modelo_bin = ruta_modelo_xml.split(".")[0] + ".bin" #Cambiar la extensión de modelo a bin 

core = IECore() #Inicializo el objeto IECore
network = core.read_network(model=ruta_modelo_xml, weights=ruta_modelo_bin) #Inicializo el objeto de tipo IENetwork a través de la función core.read_work


ResMobNet_v4 (LReLU) with single SSD head


**Nota:** En caso de no poder ejecutar el código anterior, desde el CMD de Windows o Terminal de Linux, primero inicializar las variables de entorno en el terminal y luego ejecutar el Jupyter Notebook.

####  Obtener información del los dispositivos
El siguiente código obtendrá los dispositivos disponibles para hacer inferencia con OpenVINO y la versión de sus plugins.

In [40]:
print("{} Dispositivos Compatibles OpenVINO en mi Computador {}".format(6*"#", 6*"#"))
dispositivos_compatibles = core.available_devices
print("Hay {} dispositivos disponibles para realizar la inferencia".format(len(dispositivos_compatibles)))
print(dispositivos_compatibles)
#Imprimir la versión del plugin del sistema
print("{} Versiones de los Plugins disponibles {}".format(6*"#", 6*"#"))
for dispositivo in dispositivos_compatibles:
    version = core.get_versions(dispositivo)
    descripcion = getattr(version[dispositivo], 'description')
    major = getattr(version[dispositivo], 'major')
    minor = getattr(version[dispositivo], 'minor')
    build_number = getattr(version[dispositivo], 'build_number')
    
    print("{}: {} build number: {} version: {}_{}".format(dispositivo, descripcion, build_number, major, minor ))
    


###### Dispositivos Compatibles OpenVINO en mi Computador ######
Hay 4 dispositivos disponibles para realizar la inferencia
['CPU', 'GNA', 'GPU', 'MYRIAD']
###### Versiones de los Plugins disponibles ######
CPU: MKLDNNPlugin build number: 2021.4.0-3839-cd81789d294-releases/2021/4 version: 2_1
GNA: GNAPlugin build number: 2021.4.0-3839-cd81789d294-releases/2021/4_with_GNA_LIB_VER==2 version: 2_1
GPU: clDNNPlugin build number: 2021.4.0-3839-cd81789d294-releases/2021/4 version: 2_1
MYRIAD: myriadPlugin build number: 2021.4.0-3839-cd81789d294-releases/2021/4 version: 2_1


Como se puede apreciar, en mi equipo hay cuatro dispositivos compatibles con los que puedo hacer inferencia, y la versión de los pluings es la más actual. 

####  Obtener información del modelo de Deep Learning
El siguiente código obtendrá los dispositivos disponibles para hacer inferencia con OpenVINO y la versión de sus plugins.

In [57]:
print("{} Información del modelo {}".format(6*"#", 6*"#"))
print("Nombre del modelo: {}".format(network.name))
nombre_capa_entrada = next(iter(network.input_info))
print("Nombre de la capa de entrada: {}".format(nombre_capa_entrada))
print("Formato de entrada: {}".format(network.input_info[nombre_capa_entrada].input_data.shape))
nombre_capa_salida = next(iter(network.outputs))
print("Nombre de la capa de salida: {}".format(nombre_capa_salida))
print("Formato de salida: {}".format(network.outputs[nombre_capa_salida].shape))


###### Información del modelo ######
Nombre del modelo: ResMobNet_v4 (LReLU) with single SSD head
Nombre de la capa de entrada: data
Formato de entrada: [1, 3, 320, 544]
Nombre de la capa de salida: detection_out
Formato de salida: [1, 1, 200, 7]


Los datos de la red, nos dan una referencia clara de como utilizar las entradas y las salidas del modelo, para así realizar la inferencia con los datos claros. Según la documentación del modelo `person-detection-retail-0013`[4], se comprobó facilmente que son los datos del modelo, ya que sus datos son según Intel son:: 
- Capa de Entrada
Forma del vector de entrada: `1, 3, 320, 544` en el formato `B, C, H, W`, donde:

    - B - Tamaño del Batch
    - C - Número de canales
    - H - alto de la imagen
    - W - ancho de la image

Order de color es en BGR.
- Capa de Salida
Forma del vector de salida: `1, 1, 200, 7` en el formato `1, 1, N, 7`, donde `N` es el número de bounding boxes detectaqdas. Cada detección tiene la forma `[image_id, label, conf, x_min, y_min, x_max, y_max]`, donde:

    - image_id - ID de la imagen en el Batch
    - label - ID predicción de la clase (1 - persona)
    - conf - precisión de la clase preedicha
    - (x_min, y_min) - coordenadas de la esquina superior izquierda del bounding box
    - (x_max, y_max) - coordenadas de la  esquina inferior derecha del bounding box

## Referencias

- [1] Intel. (2021, Agosto 31).Integrate the Inference Engine with Your Application. Intel. [https://docs.openvinotoolkit.org/latest/openvino_docs_IE_DG_Integrate_with_customer_application_new_API.html](https://docs.openvinotoolkit.org/latest/openvino_docs_IE_DG_Integrate_with_customer_application_new_API.html)
- [2] Intel. (2021, Agosto 31). ie_api.IECore Class Reference. Intel. [https://docs.openvinotoolkit.org/2021.4/ie_python_api/classie__api_1_1IECore.html](https://docs.openvinotoolkit.org/2021.4/ie_python_api/classie__api_1_1IECore.html)
- [3] Intel. (2021, Agosto 31). ie_api.IENetwork Class Reference. Intel. [https://docs.openvinotoolkit.org/2021.4/ie_python_api/classie__api_1_1IENetwork.html](https://docs.openvinotoolkit.org/2021.4/ie_python_api/classie__api_1_1IENetwork.html)
- [4] Intel. (2021, Agosto 31). person-detection-retail-0013. Intel. [https://docs.openvinotoolkit.org/latest/omz_models_model_person_detection_retail_0013.html](https://docs.openvinotoolkit.org/latest/omz_models_model_person_detection_retail_0013.html)
  